In [155]:
import datetime
import time

import numpy as np
import optuna
import pandas as pd
from sklearn import cluster
from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection
from sklearn import preprocessing

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim

from tqdm.auto import tqdm

In [107]:
TIMEOUT = 60 * 20

RED_DIM = 5

In [108]:
data_train = pd.read_csv("data/css_main_training.csv", sep=";")
data_test = pd.read_csv("data/css_main_test.csv", sep=";")
data_fuel = pd.read_csv("data/fuel_prices.csv", sep=";")

data_routes_train = pd.read_csv("data/css_routes_training.csv", sep=";")
data_routes_test = pd.read_csv("data/css_routes_test.csv", sep=";")

In [109]:
data_train.head()

,id_contract,expenses,duty_count,refrigerated,id_payer,id_currency,direction,temperature,load_size_type,contract_type,...,km_empty,km_nonempty,train_km,prim_train_line,train_intervals,ferry_duration,prim_ferry_line,ferry_intervals,km_total,max_weight
0,6V17,7.080449,0,1,sdv,PLN,I,+2°C,F,FTL,...,0,993.0,0,NaN,0,0,NaN,0.0,993.0,22000.0
1,3dki,6.726245,0,0,v7o,EUR,I,NaN,F,GCA,...,0,885.0,0,NaN,0,0,NaN,0.0,885.0,4824.0
2,Kir3,7.307691,0,0,gOP,EUR,I,NaN,F,FTL,...,163,404.0,983,22.0,1,0,NaN,0.0,567.0,25000.0
3,551r,6.245743,0,0,9mo,PLN,I,NaN,C,GCA,...,0,943.0,0,NaN,0,0,NaN,0.0,943.0,903.0
4,D7D0,7.318926,0,0,gOP,EUR,I,NaN,F,FTL,...,163,404.0,983,22.0,1,0,NaN,0.0,567.0,25000.0


In [110]:
data_routes_train.head()

,id_contract,id_contractor,id_vehicle,id_trailer,step,step_type,if_empty,external_fleet,kg_load_unload,kg_current,...,trailer_payload_pallets,trailer_payload_m3,trailer_cargo_hold_height,trailer_cargo_hold_width,trailer_cargo_hold_length,trailer_generator,trailer_length,trailer_width,trailer_heigth,id_trailer_type
0,6V17,1.0,0,0,1,P,1,1,22000.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6V17,1.0,0,0,2,K,0,1,-22000.0,22000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3dki,1.0,0,0,1,P,1,1,4824.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3dki,1.0,0,0,2,K,0,1,-4824.0,4824.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kir3,0.0,1,1,1,D,1,0,0.0,0.0,...,33.0,92.0,2.7,2.48,13.6,N,13.6,2.48,2.7,216.0


In [111]:
to_drop_routes = ["address", "estimated_time", "city"]
one_hot_cats_r = ["step_type", "vehicle_type", "vehicle_capacity_type", "trailer_generator"]
reduce_one_hot_cats_r = ["country_code", ]

to_drop = ["id_payer", "temperature", "route_start_datetime", "route_end_datetime", "route_start_unix",
           "route_end_unix", "date"]
ordinal_cats = ["load_size_type", "contract_type"]
one_hot_cats = ["direction"]
reduce_one_hot_cats = ["route_start_country", "last_unload_country", "first_load_country", "route_end_country",
                       "id_currency"]


In [112]:
# def preproc_routes(df, routes):
#     routes.drop(to_drop_routes, axis=1, inplace=True)
#     enc = preprocessing.OneHotEncoder()
#
#
#     routes = routes.groupby("id_contract").agg("mean")


print(data_routes_train.shape)
data_routes_train.groupby("id_contract").mean()

# data_routes_train.merge(data_train, on="id_contract")

(1189654, 60)


,id_contractor,id_vehicle,id_trailer,step,if_empty,external_fleet,kg_load_unload,kg_current,ferry,ferry_duration,...,trailer_payload_kg,trailer_payload_pallets,trailer_payload_m3,trailer_cargo_hold_height,trailer_cargo_hold_width,trailer_cargo_hold_length,trailer_length,trailer_width,trailer_heigth,id_trailer_type
id_contract,,,,,,,,,,,,,,,,,,,,,
001Y,0.0,1.000000,1.0,1.5,0.500000,0.000000,0.0,10500.000000,0.00,0.0,...,2740.0,0.0,0.0,2.65,2.47,13.46,0.00,0.00,0.00,205.0
0020,1.0,0.000000,1.0,1.5,0.500000,1.000000,0.0,11500.000000,0.00,0.0,...,24000.0,33.0,92.0,2.70,2.48,13.60,13.60,2.48,2.70,216.0
002w,1.0,0.000000,1.0,1.5,0.500000,1.000000,0.0,11500.000000,0.00,0.0,...,24000.0,33.0,92.0,2.95,2.48,13.60,13.60,2.48,2.95,220.0
003r,1.0,0.000000,0.0,2.5,0.250000,1.000000,0.0,9670.500000,0.25,157.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0043,0.0,1.000000,1.0,1.5,0.500000,0.000000,0.0,11000.000000,0.00,0.0,...,22000.0,33.0,92.0,2.63,2.45,13.55,13.55,2.45,2.63,253.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zztQ,0.0,1.000000,1.0,1.5,0.500000,0.000000,0.0,10000.000000,0.00,0.0,...,24000.0,33.0,92.0,2.70,2.48,13.60,13.60,2.48,2.70,216.0
zzwA,0.6,0.285714,1.0,4.0,0.285714,0.428571,0.0,6549.285714,0.00,0.0,...,22000.0,33.0,92.0,2.90,2.48,13.60,13.60,2.48,2.90,231.0
zzxo,1.0,1.000000,1.0,2.0,0.666667,1.000000,0.0,8000.000000,0.00,0.0,...,24000.0,33.0,92.0,2.70,2.48,13.60,13.60,2.48,2.70,216.0


In [113]:
def preproc_tree(df, routes, train=True, poly_f=False, red_dim=RED_DIM, enc_one_hot=None, enc_ordinal=None,
                 trans_l=None, verbose=False):
    if train:
        assert (enc_one_hot is None and enc_ordinal is None and trans_l is None)
    else:
        assert (enc_one_hot is not None and enc_ordinal is not None and trans_l is not None)
    if verbose:
        print("Preparing dataframes...")
    df = df.copy()
    routes = routes.copy()

    if verbose:
        print("Applying functions (nans and time related)...")
    nan_to_zero = ["prim_train_line", "prim_ferry_line"]
    for col_name in nan_to_zero:
        df[col_name] = df.apply(lambda row: 0 if np.isnan(row[col_name]) else row[col_name], axis=1)

    df["route_start_unix"] = df.apply(
        lambda row: time.mktime(datetime.datetime.fromisoformat(row["route_start_datetime"]).timetuple()), axis=1)
    df["route_end_unix"] = df.apply(
        lambda row: time.mktime(datetime.datetime.fromisoformat(row["route_end_datetime"]).timetuple()), axis=1)
    df["duration"] = df.apply(lambda row: row["route_end_unix"] - row["route_start_unix"], axis=1)
    df["date"] = df.apply(lambda row: row["route_start_datetime"].split()[0], axis=1)

    if verbose:
        print("Merging fuel...")
    df = df.merge(data_fuel, on="date")

    # if verbose:
    #     print("Merging routes...")
    # df = df.merge(routes, on="id_contract")

    print(f"Dropping colums {to_drop + to_drop_routes}...")
    df.drop(columns=to_drop, inplace=True)
    routes.drop(columns=to_drop_routes, inplace=True)

    #print(df.isna().sum())

    # if train:
    #     df.dropna(inplace=True)

    # if train:
    #     df.dropna()

    if train:
        df = df.sample(frac=1)

    X, y = df.drop(columns=["expenses"] + one_hot_cats + ordinal_cats + reduce_one_hot_cats).to_numpy(), df[
        "expenses"].to_numpy()
    R = routes.drop(columns=one_hot_cats_r + reduce_one_hot_cats_r).to_numpy()

    if verbose:
        print("Categoric (one hots without reduction and ordinal)...")
    df_one_hot_cats = df[one_hot_cats]
    routes_one_hot_cats = routes[one_hot_cats_r]
    if train:
        enc_one_hot = preprocessing.OneHotEncoder()
    X_one_hot_cats = enc_one_hot.fit_transform(df_one_hot_cats.to_numpy()).toarray()

    if train:
        enc_one_hot = preprocessing.OneHotEncoder()
    R_one_hot_cats = enc_one_hot.fit_transform(routes_one_hot_cats.to_numpy()).toarray()

    X = np.concatenate((X, X_one_hot_cats), axis=1)
    R = np.concatenate((R, R_one_hot_cats), axis=1)

    df_ordinal_cats = df[ordinal_cats]
    if train:
        enc_ordinal = preprocessing.OrdinalEncoder()
    X_ordinal_cats = enc_ordinal.fit_transform(df_ordinal_cats.to_numpy())
    X = np.concatenate((X, X_ordinal_cats), axis=1)

    if train:
        trans_l = []

    if verbose:
        print("One hot with reduction:")
    for i in range(len(reduce_one_hot_cats)):
        col_name = reduce_one_hot_cats[i]
        if verbose:
            print(f"\t preparing {col_name}...")
        if train:
            enc = preprocessing.OneHotEncoder()
            red = cluster.FeatureAgglomeration(n_clusters=red_dim)
        else:
            enc, red = trans_l[i]
        X_anc = enc.fit_transform(df[[col_name]].to_numpy()).toarray()
        X_anc = red.fit_transform(X_anc)
        X = np.concatenate((X, X_anc), axis=1)
        if train:
            trans_l.append((enc, red))

    for i in range(len(reduce_one_hot_cats_r)):
        col_name = reduce_one_hot_cats_r[i]
        if verbose:
            print(f"\t preparing {col_name}...")
        if train:
            enc = preprocessing.OneHotEncoder()
            red = cluster.FeatureAgglomeration(n_clusters=red_dim)
        else:
            enc, red = trans_l[i]
        R_anc = enc.fit_transform(routes[[col_name]].to_numpy()).toarray()
        R_anc = red.fit_transform(R_anc)
        R = np.concatenate((R, R_anc), axis=1)
        if train:
            trans_l.append((enc, red))

    #scaler = preprocessing.StandardScaler()
    #scaler.fit(X)
    #X = scaler.transform(X)

    if poly_f:
        if verbose:
            print("PolynomialFeatures...")
        poly = preprocessing.PolynomialFeatures(degree=2)
        X = poly.fit_transform(X)

    if verbose:
        print("Merging with routes...")
    Xdf = pd.DataFrame(X, columns=["id"] + [f"c{i}" for i in range(X.shape[1] - 1)])
    Rdf = pd.DataFrame(R, columns=["id"] + [f"c{i}" for i in range(R.shape[1] - 1)])
    Rdf["id"] = Rdf["id"].astype(str)
    idx = Rdf["id"]
    Rdf = Rdf.drop("id", axis=1)
    Rdf = Rdf.astype(float)

    Rdf["id"] = idx

    # print(Xdf)
    if verbose:
        print("\t grouping..")
    Rdf = Rdf.groupby("id").mean()

    # R = numpy_groupies.aggregate(R[:,0], R[:, 1:], "mean")

    if verbose:
        print("\t merging...")
    Xdf = Xdf.merge(Rdf, on="id")
    Xdf.drop("id", axis=1, inplace=True)
    Xdf = Xdf.fillna(0)

    print("Finalizing...")
    X = Xdf.to_numpy()

    print(f"DONE! Shape: {X.shape}")
    if train:
        return X, y, enc_one_hot, enc_ordinal, trans_l
    return X

In [114]:
#X, y = preproc_tree(data)


In [115]:
def validate(model, X, y, test_size=0.2):
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=test_size)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = metrics.mean_squared_error(y_test, y_pred) ** (1 / 2)
    return -score

In [116]:
def make_submit(n_estimators):
    X_train, y_train, enc_one_hot, enc_ordinal, trans_l = preproc_tree(data_train, train=True)
    X_test = preproc_tree(data_test, data_routes_test, train=False, enc_one_hot=enc_one_hot, enc_ordinal=enc_ordinal,
                          trans_l=trans_l, verbose=True)
    reg = ensemble.RandomForestRegressor(n_jobs=-1, n_estimators=n_estimators)
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    assert (len(y_pred) == len(data_test.index))
    print(y_pred)
    pd.Series(y_pred).to_csv(f"result{n_estimators}.txt", sep='\n', header=False, index=False)

In [117]:
X_train, y_train, _, _, _ = preproc_tree(data_train, data_routes_train, train=True, verbose=True)

Preparing dataframes...
Applying functions (nans and time related)...
Merging fuel...
Dropping colums ['id_payer', 'temperature', 'route_start_datetime', 'route_end_datetime', 'route_start_unix', 'route_end_unix', 'date', 'address', 'estimated_time', 'city']...
Categoric (one hots without reduction and ordinal)...
One hot with reduction:
	 preparing route_start_country...
	 preparing last_unload_country...
	 preparing first_load_country...
	 preparing route_end_country...
	 preparing id_currency...
	 preparing country_code...
Merging with routes...
	 grouping..
	 merging...
Finalizing...
DONE! Shape: (330055, 134)


In [118]:
pd.DataFrame(X_train)

,0,1,2,3,4,5,6,7,8,9,...,124,125,126,127,128,129,130,131,132,133
0,0.0,0.0,4.3380,23.0,50.940000,-1.470000,51.126311,16.978196,52.510000,-1.960000,...,0.0,0.545455,1.0,0.0,0.0,0.000000,0.272727,0.363636,0.363636,0.0
1,0.0,0.0,4.4075,23.0,51.410000,19.583700,54.460571,17.027730,51.410000,19.583700,...,0.0,1.000000,0.0,0.0,1.0,0.000000,1.000000,0.000000,0.000000,0.0
2,0.0,0.0,4.2851,44.0,54.348291,18.710000,54.257855,18.650115,54.348291,18.710000,...,0.0,0.000000,1.0,0.0,0.0,0.000000,1.000000,0.000000,0.000000,0.0
3,0.0,0.0,4.3396,23.0,54.257855,18.650115,44.432263,25.880000,54.257855,18.650115,...,0.0,1.000000,0.0,0.0,1.0,0.015152,0.500000,0.000000,0.000000,0.0
4,0.0,0.0,4.3455,23.0,53.720465,18.937464,53.120000,18.080000,53.720465,18.937464,...,0.0,1.000000,0.0,0.0,1.0,0.000000,1.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330050,0.0,0.0,4.2836,25.0,51.703523,17.933306,52.000000,-0.733232,51.071061,19.449846,...,0.0,0.363636,1.0,0.0,0.0,0.000000,0.181818,0.272727,0.545455,0.0
330051,0.0,0.0,4.3900,23.0,51.703523,17.933306,61.498021,23.760312,51.703523,17.933306,...,0.0,1.000000,1.0,0.0,0.0,0.016835,0.444444,0.000000,0.000000,0.0
330052,0.0,0.0,4.3323,23.0,51.071061,19.449846,54.348291,18.520000,51.071061,19.449846,...,0.0,1.000000,0.0,0.0,1.0,0.000000,1.000000,0.000000,0.000000,0.0
330053,0.0,0.0,4.2617,23.0,51.400000,19.583700,54.606068,18.231786,51.400000,19.583700,...,0.0,0.000000,0.0,1.0,0.0,0.000000,1.000000,0.000000,0.000000,0.0


In [119]:
# reg = ensemble.ExtraTreesRegressor(n_jobs=-1, n_estimators=200, max_depth=20, verbose=1, min_samples_leaf=5,
#                                    random_state=18)
# print(validate(reg, X_train, y_train))

In [120]:
# reg = ensemble.ExtraTreesRegressor(n_jobs=-1, n_estimators=200, max_depth=32, verbose=1, min_samples_leaf=5, random_state=18)


In [122]:
class Network(nn.Module):
    def __init__(self, data_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(data_size, 256),
            nn.ReLU(),
            nn.Linear(256, 1000),
            nn.ReLU(),
            nn.Linear(1000, 500),
            nn.Sigmoid()
        )

    def forward(self, X):
        return self.net(X)



In [ ]:
# class ActionCNNDataset(Dataset):
#
#     def __init__(self, X, ):
#
#
#     def __len__(self):
#         length = 0
#         for i in self.dataset_length:
#             length += i
#         return length
#
#     def __getitem__(self, idx):
#         if torch.is_tensor(idx):
#             idx = idx.tolist()
#
#         # Get class id and sample id, or fail
#         class_id = 0
#         while self.dataset_length[class_id] <= idx:
#             idx -= self.dataset_length[class_id]
#             class_id += 1
#
#         #data = np.load(self.root_dir+classes[class_id]+'/'+str(idx+1)+'.avi')
#         video = cv2.VideoCapture(self.root_dir+classes[class_id]+'/'+str(idx+1)+'.avi')
#         frames = []
#         #print('Getitem path', self.root_dir+classes[class_id]+'/'+str(idx+1)+'.avi')
#         for i in range(60):
#             ret, frame = video.read()
#             if not ret:
#                 #print("Broken dimensionality, not 60 frames, got", i, "frames")
#                 break
#             frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#             frames.append(np.array(Image.fromarray(frame)))
#         #print(np.array(frames).shape)
#         #try:
#         data = torch.from_numpy(np.array(frames).reshape((1,60,60,20)))
#         #except:
#         #    if idx == 0:
#         #        return self.__getitem__(1)
#         #    else:
#         #        return self.__getitem__(idx-1)
#         data = data.float()
#         label = class_id
#
#         sample = {'data': data, 'label': label}
#
#         return sample

In [157]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_train, y_train, test_size=0.2)
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)

learning_rate = 1e-3
# device = ('cuda:0' if torch.cuda.is_available() else 'cpu')
device = "cuda"

model = Network(X_train.shape[1]).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)

criterion = nn.CrossEntropyLoss().to(device)

In [129]:
torch.tensor(X_train[5]).to(device)

tensor([0.0000e+00, 1.0000e+00, 4.2970e+00, 2.3000e+01, 5.4348e+01, 1.8710e+01,
        5.4392e+01, 1.8409e+01, 5.4348e+01, 1.8710e+01, 5.4392e+01, 1.8409e+01,
        0.0000e+00, 2.8000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 2.8000e+01, 2.1000e+04, 3.6000e+03, 4.2320e+00,
        4.0880e+00, 3.1352e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
        0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00,
        1.5000e+00, 5.0000e-01, 0.0000e+00, 0.0000e+00, 1.0500e+04, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 5.0000e-01,
        0.0000e+00, 5.0000e-01, 5.4370e+

In [158]:
def train(net_model, X, y, opt, crit):
    net_model.train()
    loss_sum = 0
    outs = []

    for i in tqdm(range(X.shape[0])):
        data = torch.tensor(X[i]).to(device)
        label = torch.tensor(y[i]).to(device)
        label = label.type(torch.LongTensor).to(device)
        opt.zero_grad()
        output = net_model(data)
        # loss = metrics.mean_squared_error(label, output) ** (1/2)
        loss = crit(output, label)
        loss_sum += loss.item()
        outs.append((label, output.detach().cpu().numpy()))
        loss.backward()
        opt.step()
    outs = np.array(outs)
    acc = max(1 - metrics.mean_squared_error(outs[:, 1], outs[:, 0]) ** (1 / 2), 0)

    return loss_sum / X.shape[0], acc / X.shape[0]

for epoch in range(10):
    loss, acc = train(model, X_train, y_train, optimizer, criterion)
    optimizer.zero_grad()
    print(f"Epoch {epoch}: \n\t loss:{loss}\t accuracy: {acc}")



  0%|          | 0/35437 [00:00<?, ?it/s]

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
def objective(trial, X, y):
    n_est = trial.suggest_int("n_est", 150, 450)
    max_depth = trial.suggest_int("max_depth", 17, 30)
    #     min_leaf = trial.suggest_int("min_leaf", 2, 10)
    max_features = trial.suggest_float("max_features", 0.9, 1)
    alpha2 = trial.suggest_float("alpha2", 1e-20, 0.5, log=True)
    print(f"n_est value: {n_est}")
    reg = ensemble.ExtraTreesRegressor(n_jobs=-1, n_estimators=n_est, max_depth=max_depth, verbose=1, ccp_alpha=alpha2,
                                       min_samples_leaf=4, max_features=max_features, random_state=18)
    #result = model_selection.cross_val_score(reg, X, y, n_jobs=-1, cv=5, scoring="neg_root_mean_squared_error", verbose=10)
    result = validate(reg, X, y)
    print(f"RESULT WITH {n_est=}: {result}")
    return result

    #return validate(reg, X, y)

    #min_samples_split = trial.suggest_float("min_samples_split", 0, 1)
    #min_samples_leaf = trial.suggest_float("min_samples_leaf", 0, 0.5)
    #min_weight_fraction_leaf = trial.suggest_float("min_weight_fraction_leaf", 0, 0.5)
    #max_features = trial.suggest_float("max_features", 0, 1)
    #max_samples = trial.suggest_float("max_samples", 0, 1)
    #if min_samples_split == 0:
    #    min_samples_split = int(2)
    #if min_samples_leaf == 0:
    #    min_samples_leaf = int(1)
    #reg = ensemble.RandomForestRegressor(min_samples_split=min_samples_split,
    #                                    min_samples_leaf=min_samples_leaf,
    #                                    min_weight_fraction_leaf=min_weight_fraction_leaf,
    #                                    max_features=max_features,
    #                                  max_samples=max_samples)
    #alpha = trial.suggest_float("alpha", 0, 1)

    #max_depth = trial.suggest_int("max_depth", 1, 10)
    #print(f"suggested max_depth: {max_depth}")
    #reg = ensemble.RandomForestRegressor(max_depth=max_depth)
    #result = validate(reg)
    #print(f"result with {max_depth=}:")
    #print(result)
    #return result
    #scores = model_selection.cross_val_score(reg, X, y, cv=5, scoring="neg_root_mean_squared_error")
    #return np.mean(scores)

In [ ]:
def study_visualize(study):
    optuna.visualization.plot_optimization_history(study).show()
    optuna.visualization.plot_parallel_coordinate(study).show()
    optuna.visualization.plot_contour(study).show()
    optuna.visualization.plot_slice(study).show()
    optuna.visualization.plot_param_importances(study).show()
    best_params = study.best_trial.params
    best_value = study.best_trial.value
    print(f"best params:\n{best_params}")
    print(f"best value {best_value}")

In [ ]:
def make_optimization(study):
    X_train, y_train, _, _, _ = preproc_tree(data_train, train=True)
    study.optimize(lambda trial: objective(trial, X_train, y_train), timeout=TIMEOUT)
    study_visualize(study)
    return study

In [ ]:
# study = optuna.create_study(direction="maximize", study_name="opt_0215")
make_optimization(study)

In [ ]:
# study = optuna.create_study(direction="maximize", study_name="opt_0215")
make_optimization(study)

In [ ]:
#def initial_guess(study):
#    sampler = study.sampler
#    params = {"min_samples_split": 0.,
#              "min_samples_leaf": 0.,
#              "min_weight_fraction_leaf": 0.,
#              "max_features": 1.,
#              "max_samples": 1.,
#              }
#    study.sampler = optuna.samplers.PartialFixedSampler(params, sampler)
#    study.optimize(objective, n_trials=1)
#    study.sampler = sampler

In [ ]:
#initial_guess(study)

In [ ]:
#reg = ensemble.RandomForestRegressor(verbose=10, n_jobs=-1, n_estimators=100)
#X, y = preproc_tree(data, 5, poly_f=False)
#print(validate(reg, X, y))

In [ ]:
#200 est
#random forest after adding time and fuel
#cat reduce to 5
#time ?
-0.15119003329514882

In [ ]:
#100 est
#random forest after adding time and fuel
#cat reduce to 8
#time 3.8
-0.1514203883840837

In [ ]:
#100 est
#random forest after adding time and fuel
#cat reduce to 4
#time 3.5
-0.15207472815412973

In [ ]:
#100 est
#random forest after adding time and fuel
#cat reduce to 5
#time 4.3
-0.14865615526799092

In [ ]:
#100 est
#random forest after adding time and fuel
#cat reduce to 4
#time 3.5
-0.15207472815412973

In [ ]:
#100 est
#random forest after adding time and fuel
#cat reduce to 5
#time 4.3
-0.14865615526799092

In [ ]:
#random forest without hiperparams optimiztion
0.15468864180454378

In [ ]:
#with more cats
#only interactions
0.28300849758869806

In [ ]:
#with more cats
0.2809768534229591

In [ ]:
#with some cats
0.28220298380787107

In [ ]:
#with poly 2
#only interactions
np.mean([-0.32750984, -0.32808752, -0.32783682, -0.32724463, -0.32590006])

In [ ]:
#with poly 2
np.mean([-0.32557581, -0.33074444, -0.32652596, -0.32451948, -0.32606595])

In [ ]:
#without poly
np.mean([-0.49644549, -0.49597342, -0.49757726, -0.49581476, -0.49521693])